In [15]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [16]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GaussianNoise, GlobalMaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [17]:
#Downloading dataset
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

bone-fracture-detection-using-xrays.zip: Skipping, found more recently modified local copy (use --force to force download)
bone-fracture-detection-using-xrays.zip: Skipping, found more recently modified local copy (use --force to force download)


In [18]:
import zipfile
zip_ref = zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [19]:
#Loading the images
x_train= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

x_test= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.
Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.


In [20]:
class_names = x_train.class_names
class_names

['fractured', 'not fractured']

['fractured', 'not fractured']

In [21]:
# Normalizing
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

x_train = x_train.map(process)
x_test = x_test.map(process)

In [22]:
input = (256, 256, 3)

In [23]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model

In [24]:
# Creating the base model
base_model = InceptionV3(weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False

In [44]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.25)(x)
predictions = layers.Dense(1, activation='sigmoid')(x)

In [45]:
model = Model(inputs=base_model.input, outputs=predictions)

In [46]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [47]:
history = model.fit(x_train,epochs=10, batch_size=100, validation_data=x_test)

Epoch 1/10
89/89 [==============================] - 35s 323ms/step - loss: 0.5470 - accuracy: 0.7218 - val_loss: 0.6339 - val_accuracy: 0.6683
Epoch 2/10
89/89 [==============================] - 29s 311ms/step - loss: 0.3537 - accuracy: 0.8473 - val_loss: 0.5220 - val_accuracy: 0.7217
Epoch 3/10
89/89 [==============================] - 30s 320ms/step - loss: 0.2528 - accuracy: 0.9031 - val_loss: 0.5648 - val_accuracy: 0.7233
Epoch 4/10
89/89 [==============================] - 29s 319ms/step - loss: 0.1963 - accuracy: 0.9350 - val_loss: 0.5097 - val_accuracy: 0.7650
Epoch 5/10
89/89 [==============================] - 30s 323ms/step - loss: 0.1509 - accuracy: 0.9500 - val_loss: 0.5418 - val_accuracy: 0.7483
Epoch 6/10
89/89 [==============================] - 31s 324ms/step - loss: 0.1306 - accuracy: 0.9566 - val_loss: 0.5234 - val_accuracy: 0.7600
Epoch 7/10
89/89 [==============================] - 30s 322ms/step - loss: 0.1030 - accuracy: 0.9706 - val_loss: 0.6140 - val_accuracy: 0.7467